# Shrinkage

The subset selection methods described in the previous section involve using least squares to fit a linear model that contains a subset of the predictors. As an alternative, we can fit a model containing all p predictors using a technique that constrains or regularizes the coefficient estimates, or equivalently, that shrinks the coefficient estimates towards zero. It may not be immediately obvious why such a constraint should improve the fit, but it turns out that shrinking the coefficient estimates can significantly reduce their variance. The two best-known techniques for shrinking the regression coefficients towards zero are ridge regression and the lasso.

## Ridge regression

Recall from {doc}`Linear regression <../02-linear-reg/overview>` that the least squares fitting procedure estimates $\beta_0, \beta_1, \ldots, \beta_D$ by minimizing the residual sum of squares:

$$
\textrm{RSS} = \sum_{i=1}^N \left( y_i - \beta_0 - \sum_{j=1}^D x_{ij} \beta_j \right)^2
$$

Ridge regression is very similar to least squares, except that the coefficients are estimated by minimizing a slightly different quantity. In particular, the ridge regression coefficient estimates $\beta^R$ are the values that minimise

$$
\sum_{i=1}^N \left( y_i - \beta_0 - \sum_{j=1}^D x_{ij} \beta_j \right)^2 + \lambda \sum_{j=1}^D \beta_j^2,
$$

where $\lambda \geq 0$ is a tuning parameter. The term $\lambda \sum_{j=1}^D \beta_j^2$ is called a *shrinkage penalty* because it shrinks the coefficient estimates towards zero. The tuning parameter $\lambda$ controls the amount of shrinkage: for large values of $\lambda$, the coefficients are very strongly shrunk towards zero, whereas for small values of $\lambda$, the coefficients are barely shrunk at all. In the limit as $\lambda \rightarrow 0$, ridge regression recovers the least squares estimates.

Note that the ridge regression penalty has the effect of shrinking the coefficient estimates $\beta_j$ for all $j$, but it has no effect on $\beta_0$. This is because the penalty only includes the sum of squares of the $\beta_j$, not the $\beta_0$. In other words, the penalty has no effect on the intercept. This is a desirable property, since we usually do not want to regularize the intercept.


Watch the 9-minute video below for a visual explanation of Ridge regression:

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/Q81RR3yKn30?start=57&end=619" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining Ridge regression, by StatQuest](https://www.youtube.com/embed/Q81RR3yKn30?start=57&end=619)
```


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV

%matplotlib inline

In [ ]:
credit_url = "https://github.com/pykale/transparentML/raw/main/data/Credit.csv"

credit_df = pd.read_csv(credit_url)
credit_df["Student2"] = credit_df.Student.map({"No": 0, "Yes": 1})
credit_df["Own2"] = credit_df.Own.map({"No": 0, "Yes": 1})
credit_df["Married2"] = credit_df.Married.map({"No": 0, "Yes": 1})
credit_df["South"] = credit_df.Region.map(
    {"South": 1, "North": 0, "West": 0, "East": 0}
)
credit_df["West"] = credit_df.Region.map({"West": 1, "North": 0, "South": 0, "East": 0})
credit_df["East"] = credit_df.Region.map({"East": 1, "North": 0, "South": 0, "West": 0})
# credit_df["Region2"] = credit_df.Region.astype("category")
credit_df.head(3)

In [ ]:
X = credit_df.drop(["Own", "Student", "Married", "Region", "Balance"], axis=1).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
y = credit_df.Balance.values

In [ ]:
lambdas = np.logspace(-2, 4, 20)

coef_ridge = []

for lambda_ in lambdas:
    ridge = Ridge(alpha=lambda_)
    ridge.fit(X_scaled, y)
    coef_ridge.append(ridge.coef_)

coef_ridge = np.array(coef_ridge)

In [ ]:
plt.plot(lambdas, coef_ridge[:, 0], c="black", ls="-", label="Income")
plt.plot(lambdas, coef_ridge[:, 1], c="tab:red", ls="--", label="Limit")
plt.plot(lambdas, coef_ridge[:, 2], c="tab:blue", ls=":", label="Rating")
plt.plot(lambdas, coef_ridge[:, 6], c="orange", ls=":", label="Student")

plt.legend()
plt.xscale("log")
plt.ylim(-300, 500)
plt.xlabel(r"$\lambda$")
plt.ylabel("Standardised Coefficients")
plt.show()

## Lasso

Ridge regression does have one obvious disadvantage. Unlike best subset, forward stepwise, and backward stepwise selection, which will generally select models that involve just a subset of the variables, ridge regression will include all p predictors in the final model. The penalty λ β j 2 in (6.5) will shrink all of the coefficients towards zero, but it will not set any of them exactly to zero (unless λ = ∞). This may not be a problem for prediction accuracy, but it can create a challenge in model interpretation in settings in which the number of variables $D$ is quite large.

$$
\sum_{i=1}^N \left( y_i - \beta_0 - \sum_{j=1}^D x_{ij} \beta_j \right)^2 + \lambda \sum_{j=1}^D |\beta_j| = \textrm{RSS} + \lambda \sum_{j=1}^D |\beta_j|.
$$

Watch the 8-minute video below for a visual explanation of Lasso:

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/NGf0voTMlcs?start=15" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining Lasso, by StatQuest](https://www.youtube.com/embed/NGf0voTMlcs?start=15)
```



## Exercises

min 3 max 5

